In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [3]:
from langchain_core.tools import tool

@tool
def add_num(a:int, b:int) -> int:
    """
        add two number
        a: number a
        b: number b
        return sum of two number 
    """
    return a + b

In [12]:
add_num.invoke({"a":3, "b":4})

7

In [5]:
from langgraph.prebuilt import ToolNode

tool_nodes = ToolNode([add_num])

In [6]:
tool_calls = [{"name": "add_num", "args": {"a": 5, "b": 3}, "id": "1", "type": "tool_call"}]
result = tool_nodes.invoke(tool_calls)

In [7]:
result

{'messages': [ToolMessage(content='8', name='add_num', tool_call_id='1')]}

### call tools

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.prebuilt import create_react_agent

model = ChatGoogleGenerativeAI(model="models/gemini-2.5-flash", api_key=GEMINI_API_KEY)


agent = create_react_agent(
    model=model,
    tools=[add_num]
)

In [11]:
agent.invoke({"messages":"add 2 and 8"})

{'messages': [HumanMessage(content='add 2 and 8', additional_kwargs={}, response_metadata={}, id='9a10beb8-b660-47f4-982c-85c474733de2'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'add_num', 'arguments': '{"b": 8.0, "a": 2.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--2f5e0baa-938d-4675-8758-7974a2d32438-0', tool_calls=[{'name': 'add_num', 'args': {'b': 8.0, 'a': 2.0}, 'id': '7b3eb7ea-94b5-4059-a576-5ae932c6fa21', 'type': 'tool_call'}], usage_metadata={'input_tokens': 72, 'output_tokens': 89, 'total_tokens': 161, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 69}}),
  ToolMessage(content='10', name='add_num', id='8fd634f8-09b8-4aed-9fce-eec143c0189c', tool_call_id='7b3eb7ea-94b5-4059-a576-5ae932c6fa21'),
  AIMessage(content='The sum of 2 and 8 is 10.', additional_kwargs={}, response_metadata={'pr

In [15]:
model_with_tools = model.bind_tools([add_num])

In [22]:
response_message = model_with_tools.invoke("add 5 and 6")

In [23]:
tool_call = response_message.tool_calls[0]

In [25]:
add_num.invoke(tool_call)

ToolMessage(content='11', name='add_num', tool_call_id='c6b62162-9595-484e-8329-4ce7df3ac51a')

### Tool Customization

In [ ]:
# parameter descriptions
# Explicit input schema
from langchain_core.tools import tool
from pydantic import BaseModel, Field

class TempCustomSchema(BaseModel):
    a:int = Field(description="First operand")
    b:int = Field(description="Second operand")

@tool("add_nums", args_schema=TempCustomSchema)
def add_num(a:int, b:int) -> int:
    """
        add two numer
        a: number
        b: number
        return sum of numbers
    """
    return a+b 

### Context Management 

#### config

In [29]:
from langchain_core.tools import tool
from langchain_core.runnables import RunnableConfig

@tool("add_num")
def add_num(a:int, b:int, config:RunnableConfig)-> int:
    """"add two number

    Args:
        a: number
        b: number
    """
    print(config["configurable"].get("sample", "blank"))
    return a + b

In [30]:
from langgraph.prebuilt import ToolNode

tool_nodes = ToolNode([add_num])

In [35]:
tool_calls = [{"name": "add_num", "args": {"a": 5, "b": 3}, "id": "1", "type": "tool_call"}]

tool_nodes.invoke(tool_calls, config={"configurable": {"sample": "user_123"}})

user_123


{'messages': [ToolMessage(content='8', name='add_num', tool_call_id='1')]}

#### short-term memory

In [60]:
from langchain_core.tools import tool
from langgraph.prebuilt.chat_agent_executor import AgentState
from typing import Annotated, NotRequired
from langgraph.prebuilt import InjectedState, create_react_agent

class CustomState(AgentState):
    user_name: NotRequired[str]

@tool
def get_user_name(
    state: Annotated[CustomState, InjectedState]
) -> str:
    """Retrieve the current user-name from state."""
    # Return stored name or a default if not set
    return state.get("user_name", "Unknown user")

In [61]:


agent = create_react_agent(
    model=model,
    tools=[get_user_name],
    state_schema=CustomState
)

In [ ]:
agent.invoke({"messages": "what's my name?"})

{'messages': [HumanMessage(content="what's my name?", additional_kwargs={}, response_metadata={}, id='b9d0f8f2-060b-47a1-8d01-df778ce5b431'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_user_name', 'arguments': '{}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--f8027f1f-1bd7-4082-bdf8-8502f37ef264-0', tool_calls=[{'name': 'get_user_name', 'args': {}, 'id': '27b4db6c-498e-4cdd-ac99-768b5b859e89', 'type': 'tool_call'}], usage_metadata={'input_tokens': 39, 'output_tokens': 57, 'total_tokens': 96, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 45}}),
  ToolMessage(content='Unknown user', name='get_user_name', id='85a8cf50-a823-4d62-8a7c-717021dc5e9b', tool_call_id='27b4db6c-498e-4cdd-ac99-768b5b859e89'),
  AIMessage(content="I don't know your name.", additional_kwargs={}, response_metadata={'prompt_fe

#### long-term memory

In [64]:
from langchain_core.runnables import RunnableConfig
from langchain_core.tools import tool
from langgraph.graph import StateGraph
from langgraph.config import get_store

@tool
def get_user_info(config: RunnableConfig) -> str:
    """Look up user info."""
    # Same as that provided to `builder.compile(store=store)`
    # or `create_react_agent`
    store = get_store()
    user_id = config["configurable"].get("user_id")
    user_info = store.get(("users",), user_id)
    return str(user_info.value) if user_info else "Unknown user"

#### Tool features

In [66]:
# immediate result
@tool(return_direct=True)
def add_num(a:int, b:int):
    """add two numbers
    Args:
        a: int number
        b: int number
    """
    return a+b

In [67]:
# force tool to use
@tool(return_direct=True)
def greet(user_name: str) -> int:
    """Greet user."""
    return f"Hello {user_name}!"

tools = [greet]

configured_model = model.bind_tools(
    tools,
    # Force the use of the 'greet' tool
    tool_choice={"type": "tool", "name": "greet"}
)

In [68]:
# disable parallel call
model.bind_tools(
    tools,
    parallel_tool_calls=False
)

RunnableBinding(bound=ChatGoogleGenerativeAI(model='models/gemini-2.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002010B566850>, default_metadata=(), model_kwargs={}), kwargs={'tools': [{'type': 'function', 'function': {'name': 'greet', 'description': 'Greet user.', 'parameters': {'properties': {'user_name': {'type': 'string'}}, 'required': ['user_name'], 'type': 'object'}}}], 'parallel_tool_calls': False}, config={}, config_factories=[])

In [77]:
# handle error
from langchain_core.messages import AIMessage
from langgraph.prebuilt import ToolNode

def multiply(a: int, b: int) -> int:
    """multiply two numbers

    Args:
        a: integer number
        b: integer number
    """
    if a == 42:
        raise ValueError("The ultimate error")
    return a * b

# Default error handling (enabled by default)
tool_node = ToolNode([multiply])

message = AIMessage(
    content="",
    tool_calls=[{
        "name": "multiply",
        "args": {"a": 42, "b": 7},
        "id": "tool_call_id",
        "type": "tool_call"
    }]
)

result = tool_node.invoke({"messages": [message]})
result

{'messages': [ToolMessage(content="Error: ValueError('The ultimate error')\n Please fix your mistakes.", name='multiply', tool_call_id='tool_call_id', status='error')]}

In [76]:
# disable error handling 
# tool_node = ToolNode([multiply], handle_tool_errors=False)
# result = tool_node.invoke({"messages": [message]})
# result

In [79]:
# custom error messages
tool_node = ToolNode(
    [multiply],
    handle_tool_errors="Can't use 42 as the first operand, please switch operands!"
)

result = tool_node.invoke({"messages": [message]})
result

{'messages': [ToolMessage(content="Can't use 42 as the first operand, please switch operands!", name='multiply', tool_call_id='tool_call_id', status='error')]}

#### prebuild tool